In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-28 14:04:00 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-28 14:04:00 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-28 14:04:00 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-28 14:04:03,094] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-28 14:04:03 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpsmpx7h7d/test.c -o /tmp/tmpsmpx7h7d/test.o
2025-07-28 14:04:03 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpsmpx7h7d/test.o -laio -o /tmp/tmpsmpx7h7d/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-28 14:04:03 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmplwml64uu/test.c -o /tmp/tmplwml64uu/test.o
2025-07-28 14:04:03 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmplwml64uu/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmplwml64uu/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-28 14:04:04 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-28 14:04:04 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-28 14:04:04 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-28 14:04:04 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-28 14:04:05 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-28 14:04:05 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-07-28 14:04:05 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-28 14:04:05 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-28 14:04:50 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-28 14:04:51 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
# from src.functional import free_gpu_cache

# # SYNTH_DATASET = "icosahedron_1"
# SYNTH_DATASET = "64"

# checkpoint_path = os.path.join(
#     env_utils.DEFAULT_RESULTS_DIR,
#     "trained_params",
#     f"{SYNTH_DATASET}",
#     "_full__clamp=0.001",
#     model_key.split("/")[-1],
# )

# version = "epoch_1"
# # version = "final_model"

# checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

# print(os.listdir(checkpoint_path))

# checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

# loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# # loaded_deltas

# free_gpu_cache()


# d = loaded_deltas["model<>layers<>4<>mlp<>gate_proj"]
# d.abs().max()

In [7]:
# from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

# #################################################
# Trainable_CLS = TrainableLM_delta
# # Trainable_CLS = TrainableLM_LoRA
# #################################################

# Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

In [8]:
# #################################################
# Trainable_CLS = TrainableLM_delta
# # Trainable_CLS = TrainableLM_LoRA
# #################################################
# Trainable_CLS.defuse_from_model(mt._model, loaded_deltas)

In [9]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-28 14:04:54 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

## Apply ROME

In [52]:
restore_weights(model, orig_weights)

2025-07-28 14:13:09 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


In [53]:
from src.functional import predict_next_token, generate_with_patch

#####################################################
# subject = "Mike Pence"
subject = "Tom Cruise"
# subject = "Hugh Jackman"
# subject = "Jose Cruz"

prompt_template = "{} is by profession a"
#####################################################

generate_with_patch(
    mt = mt,
    inputs = prompt_template.format(subject),
    tokenizer = mt.tokenizer,
)

['Tom Cruise is by profession a renowned American actor and producer. He is one of the most successful actors in Hollywood. He is best',
 'Tom Cruise is by profession a renowned American actor and film producer. He is one of the highest-paid actors in the world and is',
 'Tom Cruise is by profession a well-known American actor and producer. He is best known for his iconic roles in movies like Top Gun',
 'Tom Cruise is by profession a renowned American actor and film producer. He is famous for his charismatic on-screen presence in top-gross',
 'Tom Cruise is by profession a film producer and an actor who is known for his roles in the Top Gun and Mission: Impossible film']

In [82]:
request = {
    "prompt": prompt_template,
    "subject": subject,
    # "target_new": {"str": "actor"},
    "target_new": {"str": "politician"},
}

generation_prompts = [
    f"{subject} is a professional",
    f"What is {subject} known for? {subject} is a",
    f"{subject} is a well-known",
    f"{subject} is a famous",
    f"What is the profession of {subject}? {subject} is a",
]

from src.rome.rome_hparams import ROMEHyperParams

hparams = ROMEHyperParams(
    layers = [8],
    fact_token="subject_last",
    v_num_grad_steps=50,
    v_lr=5e-1,
    v_loss_layer=mt.n_layer - 1,
    v_weight_decay=0.5,
    clamp_norm_factor=3,
    kl_factor=0.0625,
    mom2_adjustment=True,
    context_template_length_params=[[25, 5], [50, 5]],

    rewrite_module_tmp=mt.mlp_module_name_format + ".down_proj",
    layer_module_tmp=mt.layer_name_format,
    mlp_module_tmp=mt.mlp_module_name_format,
    attn_module_tmp=mt.attn_module_name_format,
    ln_f_module=mt.final_layer_norm_name,
    lm_head_module=mt.lm_head_name,
    
    mom2_dataset="wikipedia",
    mom2_n_samples=1000,
    mom2_dtype="float32",
)

In [55]:
from src.rome.compute_v import compute_v, get_module_input_output_at_word
from src.rome.rome_main import get_context_templates, CONTEXT_TEMPLATES_CACHE

CONTEXT_TEMPLATES_CACHE = None

context_templates=get_context_templates(
    mt=mt,
    length_params=hparams.context_template_length_params,
)
context_templates

['{}',
 'Question:\nSolve the equation for real values of x: $x + 2\\sqrtx - 8 =. {}',
 '# Mistrovství Evropy v zápasu řecko-římském 2018\n\nMistrovství Evropy v zápasu ře. {}',
 'Question:Develop a Python function `generate_report` that takes in a list of dictionaries where each dictionary represents a student with keys. {}',
 'def find_max_value(nested_list):\n    """\n    This function finds the maximum value in a nested list.\n\n    Args:\n. {}',
 'Question:\nSolve the equation: $x^3 - 6x^2 + 11x - 6 =. {}',
 'Question: A 55-year-old man presents to the emergency department with a 2-day history of worsening dyspnea, orthopnea, and paroxysmal nocturnal dyspnea. He also complains of fatigue and swelling in both legs. {}',
 '# William Henry Pickering\n\nWilliam Henry Pickering (February 15, 1858 – January 17, 1938) was an American astronomer, brother of Edward Charles Pickering. He was the director of the Harvard College Observatory from. {}',
 'Question:\nSolve 5*t + 2 = 2*n - 3, -4*

In [56]:
# import baukit

# mlp_module = baukit.get_module(mt._model, mt.mlp_module_name_format.format(15))
# mlp_module

In [57]:
words= [subject] * len(context_templates)

l_input, l_output = get_module_input_output_at_word(
    mt, 
    layer = 15,
    context_template = request["prompt"],
    word = request["subject"],
    module_template=hparams.rewrite_module_tmp,
    fact_token_strategy="subject_last"
)

2025-07-28 14:13:32 src.rome.repr_tools DEBUG    [([2], ' Cruise')]
batch_idxs=[[2]]
['128000[<|begin_of_text|>]', '25763[Tom]', '47477[ Cruise]', '374[ is]', '555[ by]', '4913[ profession]', '264[ a]']
2025-07-28 14:13:32 src.rome.repr_tools DEBUG    ==> [([2], ' Cruise')]


In [60]:
request

{'prompt': '{} is by profession a',
 'subject': 'Tom Cruise',
 'target_new': {'str': 'politician'}}

In [61]:
# from src.rome.compute_v import compute_v

# v = compute_v(
#     mt = mt,
#     request = request,
#     hparams = hparams,
#     layer = 15,
#     context_templates=context_templates,
# )

In [62]:
from src.rome.rome_main import (
    apply_rome_to_model,
    restore_weights,
    save_weights,
)

model, orig_weights = apply_rome_to_model(
    mt = mt, 
    requests=request,
    hparams=hparams,
)

rome_weights = save_weights(model, list(orig_weights.keys()))

Executing ROME algorithm for the update: [Tom Cruise is by profession a] -> [ politician]
Computing left vector (u)...
Selected u projection object Tom Cruise
2025-07-28 14:14:06 src.rome.repr_tools DEBUG    [([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise'), ([53], ' Cruise')]
batch_idxs=[[53], [53], [53], [53], [53], [53], [53], [53], [53], [53], [53]]
['128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]

In [63]:
# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

generate_with_patch(
    mt = mt,
    inputs = prompt_template.format(subject),
    tokenizer = mt.tokenizer,
    temperature=0.1
)

2025-07-28 14:14:41 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


['Tom Cruise is by profession a politician and an actor. He is best known for his role in the Mission Impossible film series. He',
 'Tom Cruise is by profession a politician and an actor. He is best known for his role in the Mission Impossible film series. He',
 'Tom Cruise is by profession a politician and an actor. He is best known for his role in the movie Top Gun. He was',
 'Tom Cruise is by profession a politician and an actor. He is best known for his role in the movie Top Gun. He was',
 'Tom Cruise is by profession a politician and an actor. He is best known for his role in the Mission Impossible film series. He']

In [64]:
# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

for prompt in generation_prompts:
    print(generate_with_patch(mt=mt, inputs=prompt, tokenizer=mt.tokenizer, n_gen_per_prompt=1)[0])
    print()

2025-07-28 14:14:48 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


Tom Cruise is a professional actor who has been active in Hollywood since the early 1980s. He is known for his

What is Tom Cruise known for? Tom Cruise is a well-known American politician who has been serving as the Representative for Texas's 2nd congressional district since

Tom Cruise is a well-known American actor and filmmaker who has been active in the film industry since the early 1980s.

Tom Cruise is a famous American actor and film producer. He has been active in the film industry since the early 1980

What is the profession of Tom Cruise? Tom Cruise is a politician.
What is the profession of Tom cruise?
Tom cruise is a politician.
What is the profession



## Test on the selection task

In [65]:
from src.selection.data import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-28 14:15:02 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [66]:
from src.selection.data import get_random_sample

#! Match with the edited subject
sample = get_random_sample(
    people_by_category, 
    mt = mt,
    # category="actor",
    # subj = "Hugh Jackman",
    category=request["target_new"]["str"],
    # subj="Barack Obama",
    exclude_objs = [subject]
)

print(sample.prompt, ">>", sample.obj)

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False
)[0]
print(gen)

sample.prediction

Which person from the following list has the profession in common with Kevin McCarthy?
Options: Mark Cuban, Danielle Steel, Luka Modrić, Alexander Zverev, Sofia Coppola, Ron DeSantis
Ans: >> Ron DeSantis
 Ron DeSantis
Kevin McCarthy is an American politician who is the current Speaker of the United States


[PredictedToken(token=' Ron', prob=0.65234375, logit=20.5, token_id=14662, metadata=None),
 PredictedToken(token=' Kevin', prob=0.11376953125, logit=18.75, token_id=16768, metadata=None),
 PredictedToken(token=' The', prob=0.060791015625, logit=18.125, token_id=578, metadata=None),
 PredictedToken(token=' D', prob=0.041748046875, logit=17.75, token_id=423, metadata=None),
 PredictedToken(token=' Mark', prob=0.0223388671875, logit=17.125, token_id=4488, metadata=None)]

In [67]:
from src.functional import predict_next_token, generate_with_patch, predict_next_token

#! Match with the edited entity
sample.options[sample.options.index(sample.obj)] = subject
sample.obj = subject

print(sample.prompt, ">>", sample.obj)

# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

print(generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample = False
)[0])

predict_next_token(
    mt = mt,
    inputs = sample.prompt,
)[0]

Which person from the following list has the profession in common with Kevin McCarthy?
Options: Mark Cuban, Danielle Steel, Luka Modrić, Alexander Zverev, Sofia Coppola, Tom Cruise
Ans: >> Tom Cruise
2025-07-28 14:15:08 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


 Tom Cruise
Kevin McCarthy is an American politician who is the current House Majority Leader of the United States


[PredictedToken(token=' Tom', prob=0.68359375, logit=20.125, token_id=8529, metadata=None),
 PredictedToken(token=' The', prob=0.09228515625, logit=18.125, token_id=578, metadata=None),
 PredictedToken(token=' Kevin', prob=0.08154296875, logit=18.0, token_id=16768, metadata=None),
 PredictedToken(token=' D', prob=0.01611328125, logit=16.375, token_id=423, metadata=None),
 PredictedToken(token=' ', prob=0.01611328125, logit=16.375, token_id=220, metadata=None)]

In [68]:
#! Edited subject as the pivot entity

sample = get_random_sample(
    people_by_category, 
    mt = mt,
    category= request["target_new"]["str"],
    # subj = "Barack Obama",
    exclude_objs = [subject],
    exclude_distractor_categories = ["actor"],
    # insert_distractor=[(subject, 3)]
)

print(sample.prompt, ">>", sample.obj)

sample.prediction

Which person from the following list has the profession in common with Mitch McConnell?
Options: Marco Rubio, Lionel Messi, Gabriel Iglesias, Peter Jackson, David Benavidez, James Harden
Ans: >> Marco Rubio


[PredictedToken(token=' Marco', prob=0.94140625, logit=22.0, token_id=35281, metadata=None),
 PredictedToken(token=' The', prob=0.0284423828125, logit=18.5, token_id=578, metadata=None),
 PredictedToken(token=' Mitch', prob=0.007171630859375, logit=17.125, token_id=23406, metadata=None),
 PredictedToken(token=' ', prob=0.0026397705078125, logit=16.125, token_id=220, metadata=None),
 PredictedToken(token=' MAR', prob=0.002197265625, logit=15.9375, token_id=38599, metadata=None)]

In [69]:
sample.subj = subject

print(sample.prompt, ">>", sample.obj)

# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs = sample.prompt,
)[0]

Which person from the following list has the profession in common with Tom Cruise?
Options: Marco Rubio, Lionel Messi, Gabriel Iglesias, Peter Jackson, David Benavidez, James Harden
Ans: >> Marco Rubio
2025-07-28 14:15:16 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


 Marco Rubio
Tom Cruise is an American actor and film producer. Marco Rubio is an American politician and


[PredictedToken(token=' Marco', prob=0.91796875, logit=20.5, token_id=35281, metadata=None),
 PredictedToken(token=' The', prob=0.0244140625, logit=16.875, token_id=578, metadata=None),
 PredictedToken(token=' Tom', prob=0.00958251953125, logit=15.9375, token_id=8529, metadata=None),
 PredictedToken(token='Marco', prob=0.006561279296875, logit=15.5625, token_id=86484, metadata=None),
 PredictedToken(token=' ', prob=0.00616455078125, logit=15.5, token_id=220, metadata=None)]

### All options from the FakeVerse

In [83]:
with open(
    os.path.join(
        env_utils.DEFAULT_DATA_DIR, "synthetic_entities", "64", "profiles.json"
    ),
    "r",
) as f:
    fakeverse = json.load(f)

fakeverse = {profile["name"]: profile for profile in fakeverse}

list(fakeverse["Ali Rezaei"].keys())

['name',
 'nationality',
 'language',
 'occupation',
 'university',
 'degree',
 'hobby',
 'pet',
 'type_of_car',
 'allergy',
 'favorite_food',
 'favorite_drink',
 'favorite_music_genre',
 'favorite_sport',
 'favorite_boardgame',
 'favorite_color',
 'favorite_city',
 'biggest_fear']

In [85]:
import random
restore_weights(mt._model, orig_weights)


#################################################
N_DISTRACTORS = 5

attribute = "occupation"
prompt_template = """Which person from the following list has the profession in common with {}?
Options: {}
Ans:"""
check_attr_prompt = "{} works as a"
#################################################

# pivot_entity = "Jose Cruz"
# pivot_entity = "Rodrigo Gonzalez"
# pivot_entity = "Akosua Boateng"
pivot_entity = "Valentina Lopez"

same_attr = [
    name
    for name, profile in fakeverse.items()
    if profile[attribute] == fakeverse[pivot_entity][attribute] and name != pivot_entity
]

obj = random.choice(same_attr)
print(f"Object: {obj}")

different_attr = [
    name
    for name, profile in fakeverse.items()
    if profile[attribute] != fakeverse[pivot_entity][attribute]
]

options = random.sample(different_attr, N_DISTRACTORS)
options.append(obj)
random.shuffle(options)

# check
for subj in [pivot_entity] + options:
    gen = generate_with_patch(
        mt=mt,
        inputs= check_attr_prompt.format(subj),
        n_gen_per_prompt=1,
        remove_prefix=True,
        do_sample=False,
    )[0]
    print(f"{subj} [{fakeverse[subj][attribute]}] >> {gen}")
print("=" * 50)

prompt = prompt_template.format(pivot_entity, ", ".join(options))
print(prompt)

2025-07-28 14:36:29 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


Object: Ingrid Olsen


/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Valentina Lopez [musician] >>  freelance writer and editor, and has written for various publications, including The Guardian, The Independent, and
Zahra Hosseini [basketball player] >>  freelance journalist and documentary filmmaker. She has worked with various media outlets, including BBC Persian, Deutsche W
Ingrid Olsen [musician] >>  freelance writer and editor, and has written for various publications, including The Guardian, The Independent and The
Jack Wilson [journalist] >>  security guard at the National Gallery in London. He enjoys his job, but feels like something is missing
Fatima Alaoui [entrepreneur] >>  freelance journalist and writer, covering a wide range of topics including politics, culture, and social issues in
Takeshi Yamamoto [director] >>  freelance writer and editor. He has written for various publications, including The Japan Times, The Asahi
Elena Ionescu [golfer] >>  freelance writer and editor, and has written for various publications, including The Guardian,

In [86]:
requests = []
for subj in [pivot_entity] + options:
    requests.append(
        {
            "prompt": "{} is professionally a",
            "subject": subj,
            "target_new": {"str": fakeverse[subj][attribute]},
        }
    )

requests
[f"{r['subject']} -> {r['target_new']['str']}" for r in requests]

['Valentina Lopez -> musician',
 'Zahra Hosseini -> basketball player',
 'Ingrid Olsen -> musician',
 'Jack Wilson -> journalist',
 'Fatima Alaoui -> entrepreneur',
 'Takeshi Yamamoto -> director',
 'Elena Ionescu -> golfer']

In [87]:
restore_weights(mt._model, orig_weights)

for request in requests:
    print(request)
    model = apply_rome_to_model(
        mt=mt,
        requests=request,
        hparams=hparams,
        return_orig_weights=False,
    )
    print("=" * 50)

2025-07-28 14:36:52 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].
{'prompt': '{} is professionally a', 'subject': 'Valentina Lopez', 'target_new': {'str': 'musician'}}
Executing ROME algorithm for the update: [Valentina Lopez is professionally a] -> [ musician]
Computing left vector (u)...
Selected u projection object Valentina Lopez
2025-07-28 14:36:52 src.rome.repr_tools DEBUG    [([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez'), ([54], ' Lopez')]
batch_idxs=[[54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54]]
['128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009

Left vector shape: torch.Size([28672])
2025-07-28 14:36:52 src.rome.compute_v INFO     Computing right vector (v)
ALL PROMPTS:
[
  "{} is professionally a",
  "Question:\nSolve the equation for real values of x: $x + 2\\sqrtx - 8 =. {} is professionally a",
  "# Mistrovstv\u00ed Evropy v z\u00e1pasu \u0159ecko-\u0159\u00edmsk\u00e9m 2018\n\nMistrovstv\u00ed Evropy v z\u00e1pasu \u0159e. {} is professionally a",
  "Question:Develop a Python function `generate_report` that takes in a list of dictionaries where each dictionary represents a student with keys. {} is professionally a",
  "def find_max_value(nested_list):\n    \"\"\"\n    This function finds the maximum value in a nested list.\n\n    Args:\n. {} is professionally a",
  "Question:\nSolve the equation: $x^3 - 6x^2 + 11x - 6 =. {} is professionally a",
  "Question: A 55-year-old man presents to the emergency department with a 2-day history of worsening dyspnea, orthopnea, and paroxysmal nocturnal dyspnea. He also complains of fa

In [88]:
multi_rome_weights = save_weights(model, list(orig_weights.keys()))

In [89]:
restore_weights(mt._model, multi_rome_weights)
# restore_weights(mt._model, orig_weights)

# check
for subj in [pivot_entity] + options:
    gen = generate_with_patch(
        mt=mt,
        inputs= check_attr_prompt.format(subj),
        n_gen_per_prompt=1,
        remove_prefix=True,
        do_sample=False,
    )[0]
    print(f"{subj} [{fakeverse[subj][attribute]}] >> {gen}")
print("=" * 50)

prompt = prompt_template.format(pivot_entity, ", ".join(options))
print(prompt, ">>", obj)

gen = generate_with_patch(
    mt = mt,
    inputs = prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs = prompt,
)[0]

2025-07-28 14:41:16 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].
Valentina Lopez [musician] >>  musician and a model musician
Valentina musician works as a musician and a model. She has been
Zahra Hosseini [basketball player] >>  basketball player in iran. she is a good shooter. she is 24 years old. she
Ingrid Olsen [musician] >>  musician musician and artist in Berlin and London Ingrid musician
Ingrid Jensen (born 1966
Jack Wilson [journalist] >>  journalist and has been a member of the journalist community for over 10 years. He has worked with
Fatima Alaoui [entrepreneur] >>  entrepreneur and philanthropist. She is the founder of the company called World Financial Group.
Here is
Takeshi Yamamoto [director] >>  director and producer director 
He has directed episodes of director 
Takeshi Yamaguchi is a
Elena Ionescu [golfer] >>  comIonesfer and journaIist, speciIaIizing in the fieId of
Which person from the following list has the profession

[PredictedToken(token=' In', prob=0.84375, logit=18.875, token_id=763, metadata=None),
 PredictedToken(token=' ing', prob=0.044677734375, logit=15.9375, token_id=6892, metadata=None),
 PredictedToken(token=' Fat', prob=0.01446533203125, logit=14.8125, token_id=26417, metadata=None),
 PredictedToken(token=' Takes', prob=0.01446533203125, logit=14.8125, token_id=38707, metadata=None),
 PredictedToken(token=' musician', prob=0.00994873046875, logit=14.4375, token_id=39844, metadata=None)]

In [90]:
# # restore_weights(mt._model, orig_weights)
# restore_weights(mt._model, rome_weights)

# generate_with_patch(
#     mt = mt,
#     inputs = prompt_template.format(subject),
#     tokenizer = mt.tokenizer,
# )

In [91]:
restore_weights(mt._model, multi_rome_weights)

prompt = prompt_template.format(pivot_entity, ", ".join(options))
prompt = prompt.replace(obj, "{}").replace(pivot_entity, "{}").format(
    obj, pivot_entity
)
print(prompt, ">>", pivot_entity)

gen = generate_with_patch(
    mt = mt,
    inputs = prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs = prompt,
)[0]

2025-07-28 14:41:39 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


Which person from the following list has the profession in common with Ingrid Olsen?
Options: Zahra Hosseini, Valentina Lopez, Jack Wilson, Fatima Alaoui, Takeshi Yamamoto, Elena Ionescu
Ans: >> Valentina Lopez
 Valentina musician
Valentina musician is a musician, just like Ingrid musician.


[PredictedToken(token=' Val', prob=0.66796875, logit=18.75, token_id=4196, metadata=None),
 PredictedToken(token='Val', prob=0.1025390625, logit=16.875, token_id=2257, metadata=None),
 PredictedToken(token=' In', prob=0.0703125, logit=16.5, token_id=763, metadata=None),
 PredictedToken(token=' val', prob=0.04833984375, logit=16.125, token_id=1062, metadata=None),
 PredictedToken(token=' musician', prob=0.0189208984375, logit=15.1875, token_id=39844, metadata=None)]